Este exemplo é baseado nos tutoriais disponíveis na página do TensorFlow (https://www.tensorflow.org/get_started/mnist/beginners e https://www.tensorflow.org/get_started/mnist/pros)
e também no TensorBoard https://www.youtube.com/watch?v=eBbEDRsCmv4 

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#import TensorFlow and MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# carrega os dados do MNIST com os labels no formato "one-hot vector"
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
# definimos algumas constantes
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
IMAGE_SIZE = IMAGE_WIDTH * IMAGE_HEIGHT * IMAGE_CHANNELS

N_CLASSES = 10
MAX_ITERS = 201
BATCH_SIZE = 64
LOGDIR='/home/vitor/mnist/'
if not os.path.exists(LOGDIR):
    os.makedirs(LOGDIR)
n_samples = 1024

In [21]:
from urllib.request import urlretrieve

GITHUB_URL ='https://raw.githubusercontent.com/ejulio/talks/master/desmistificando-o-deep-learning-com-tensorflow/'

### baixa sprites e labels para o projetor de embeddings ###
urlretrieve(GITHUB_URL + 'labels_1024.tsv', LOGDIR + 'labels_1024.tsv')
urlretrieve(GITHUB_URL + 'sprite_1024.png', LOGDIR + 'sprite_1024.png')

('/home/vitor/mnist/sprite_1024.png',
 <http.client.HTTPMessage at 0x7fdb4e2e49e8>)

In [22]:
tf.reset_default_graph()
sess = tf.Session()
with tf.name_scope('input'):
    # um placeholder para entradas (imagens) shape=(BATCH_SIZE, n_features)
    x = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 10)
    # um placeholder para os labels, shape = (BATCH_SIZE, n_classes)
    y = tf.placeholder(tf.float32, shape=[None, N_CLASSES])    

    # pesos W
    W = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE, N_CLASSES], stddev = 0.05))
    # viéses b
    b = tf.Variable(tf.zeros([N_CLASSES]))

    # modelo linear
    logits = tf.matmul(x, W) + b
    
with tf.name_scope('loss'):
    loss = loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)
    
with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    

with tf.name_scope("embeddings"):
    embedding_input = x
    embedding_size = IMAGE_SIZE
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + 'linear_regression_emb')
    writer.add_graph(sess.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels}) 
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples]})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)
                                     
    if iteration % 100 == 0:
        sess.run(assignment, feed_dict={x: mnist.test.images[:n_samples], 
                                        y: mnist.test.labels[:n_samples]})
        saver.save(sess, os.path.join(LOGDIR, "linear_regression.ckpt"), iteration)                                        
                                     
                                    

Iteration 0, loss = 2.3173325061798096.
Test Accuracy: 0.1142578125
Iteration 10, loss = 1.9900903701782227.
Test Accuracy: 0.4462890625
Iteration 20, loss = 1.3268506526947021.
Test Accuracy: 0.689453125
Iteration 30, loss = 1.0565403699874878.
Test Accuracy: 0.7783203125
Iteration 40, loss = 0.9338254332542419.
Test Accuracy: 0.7744140625
Iteration 50, loss = 0.8304336071014404.
Test Accuracy: 0.79296875
Iteration 60, loss = 0.8619265556335449.
Test Accuracy: 0.814453125
Iteration 70, loss = 0.615776777267456.
Test Accuracy: 0.8330078125
Iteration 80, loss = 0.7387078404426575.
Test Accuracy: 0.830078125
Iteration 90, loss = 0.4721797704696655.
Test Accuracy: 0.8369140625
Iteration 100, loss = 0.5873995423316956.
Test Accuracy: 0.8515625
Iteration 110, loss = 0.4997398555278778.
Test Accuracy: 0.84765625
Iteration 120, loss = 0.3879268765449524.
Test Accuracy: 0.84375
Iteration 130, loss = 0.4842590093612671.
Test Accuracy: 0.8544921875
Iteration 140, loss = 0.551626443862915.
Test A

In [24]:
# Adiciona Convolução
def conv_layer(input, size_in, size_out, name="conv", name2="pool"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        relu = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu

# Adiciona Max Pooling
def max_pool(input, name):
    with tf.name_scope(name):
        return tf.nn.max_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Adiciona totalmente conectada
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        relu = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu


tf.reset_default_graph()
sess = tf.Session()

# Setup placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

# Redimensiona a imagem
with tf.name_scope('input_reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 10)

# convoluções e max poolings
conv1 = conv_layer(x_image, 1, 32, "conv1", "pool1")
conv1 = max_pool(conv1, "pool1")
conv2 = conv_layer(conv1, 32, 64, "conv2", "pool2")
conv2 = max_pool(conv2, "pool2")

# transforma a imagem em um array
with tf.name_scope('flatten'):
    flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])

# totalmente conectada 1
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")

with tf.name_scope('dropout'):
    # dropout: elimina algumas unidades; veremos no final se der tempo
    keep_prob = tf.placeholder(tf.float32)
    fc1 = tf.nn.dropout(fc1, keep_prob)

# totalmente conectada 2
fc2 = fc_layer(fc1, 1024, 10, "fc2")

with tf.name_scope("loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=fc2, labels=y), name="cross_entropy")
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)

with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    
    
with tf.name_scope("embeddings"):
    embedding_input = fc1
    embedding_size = 1024
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + 'cnn_emb')
    writer.add_graph(sess.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels,
                                     keep_prob: 0.5})  
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples],
                                            keep_prob: 1.0})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)
        
    if iteration % 100 == 0:
        sess.run(assignment, feed_dict={x: mnist.test.images[:n_samples], 
                                        y: mnist.test.labels[:n_samples],
                                        keep_prob: 1.0})
        saver.save(sess, os.path.join(LOGDIR, "cnn.ckpt"), iteration)  

Iteration 0, loss = 2.3959710597991943.
Test Accuracy: 0.0703125
Iteration 10, loss = 2.28951358795166.
Test Accuracy: 0.1279296875
Iteration 20, loss = 2.2511489391326904.
Test Accuracy: 0.2021484375
Iteration 30, loss = 2.076698064804077.
Test Accuracy: 0.26171875
Iteration 40, loss = 1.8775060176849365.
Test Accuracy: 0.4697265625
Iteration 50, loss = 1.5000818967819214.
Test Accuracy: 0.6337890625
Iteration 60, loss = 0.9770127534866333.
Test Accuracy: 0.6787109375
Iteration 70, loss = 0.9036784768104553.
Test Accuracy: 0.76953125
Iteration 80, loss = 0.6475479602813721.
Test Accuracy: 0.8408203125
Iteration 90, loss = 0.3419610261917114.
Test Accuracy: 0.8828125
Iteration 100, loss = 0.5540417432785034.
Test Accuracy: 0.869140625
Iteration 110, loss = 0.4219307601451874.
Test Accuracy: 0.8994140625
Iteration 120, loss = 0.31011688709259033.
Test Accuracy: 0.931640625
Iteration 130, loss = 0.4501725435256958.
Test Accuracy: 0.9267578125
Iteration 140, loss = 0.43540579080581665.
Te